This notebook is used mainly for the IBM capstone Project

In [2]:
 print('Hello Capstone Project Course')

Hello Capstone Project Course


# Intro
The purpose of this assignment is to explore, segment, and cluster the neighborhoods in the city of Toronto.
In order to begin the exploration of the neighborhoods, the data has to be collected, stored and cleaned. The wikipedia page: List of postal codes of Canada: M contains a list of postal codes with the first letter M. These postal codes are located within the city of Toronto. The table provided will serve as our source.
## SCRAPING WIKIPEDIA'S TABLE 
We will be using Beautifulsoup to extract the table of all neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request
from bs4 import BeautifulSoup # to parse the website files
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
 


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = requests.get(url)

soup = BeautifulSoup(request.text, 'lxml')
# print(soup.prettify()) to find table

In [3]:
table = soup.find('table',{'class':'wikitable sortable'}) # find table 
ths=table.find_all('th') # Search through the tables for the with the headings we want.



In [4]:
# Search through the tables for the headings we want and save as list
column_name=[]
for th in ths:
    column_name.append(th.text.strip())
print(column_name)


['Postal Code', 'Borough', 'Neighborhood']


In [5]:
# Search through the tables for the rows and cells we want and savepd as list
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text.strip())
    output_rows.append(output_row)
table=output_rows[1:]

In [6]:
df=pd.DataFrame(table, columns =column_name) # convert table to DF

In [7]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Data Cleansing
We are droping all Borough's not assigned, and investigating if there are Neighborhoods not assigned

In [8]:
df = df[df.Borough != 'Not assigned'] #drop rows where Borough is not assigned


In [9]:
df[df.Neighborhood == 'Not assigned'] #no Neighborhoods that exists that are 'Not assigned'


,Postal Code,Borough,Neighborhood


In [10]:
df.shape

(103, 3)

# ENRICHING THE DATA
1. We are reseting the index so that we can iterate over the dataframe
2. In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. We append the lat and long coordinates using pgeocode

In [11]:
df.reset_index(inplace=True, drop=True) #reset the index and drop the old one

In [12]:
# lat/long is not in the format Folium uses
#import pgeocode to locate lat long !pip install pgeocode if module is not found
import pgeocode
nomi = pgeocode.Nominatim('ca') 

#loop over postal codes and append lat, long on 

postal_code = df['Postal Code']
lat=[]
long=[]
count=0

for latlong in postal_code:    
    location = nomi.query_postal_code(postal_code[count])
    lat.append(location.latitude)
    long.append(location.longitude)
    count+=1

In [13]:

#convert lat long lists to dataframe and add to df
df['Longitude']=pd.DataFrame(lat) 
df['Latitude']=pd.DataFrame(long)

In [14]:
# df.drop('lat', inplace=True, axis=1) accidently created another lat, so dropping this column

In [15]:
df.head()

,Postal Code,Borough,Neighborhood,Longitude,Latitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [16]:
df['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [17]:
# we only want to work within Boroughs that contain the word "Toronto" so let' drop all Borough's that do not contain 'Toronto'
df=df[df.Borough.str.contains("Toronto")]  #drop rows where Borough does not have 'Toronto'

df.head()

,Postal Code,Borough,Neighborhood,Longitude,Latitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
19,M4E,East Toronto,The Beaches,43.6784,-79.2941


# lets add the csv version of lat/long  as well for comparison
pgeocode does not work well with folium as it does not capture the full coordinates

In [19]:
df_csv=pd.read_csv('http://cocl.us/Geospatial_data')

In [20]:
df_csv = df_csv.set_index('Postal Code') #set the index to postal code for mapping
df.set_index('Postal Code', inplace=True, drop=True)  #set the index to postal code for mapping

In [21]:
df['csv_lat']=df_csv['Latitude']
df['csv_long']=df_csv['Longitude']

In [22]:
df.head()

,Borough,Neighborhood,Longitude,Latitude,csv_lat,csv_long
Postal Code,,,,,,
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,43.654260,-79.360636
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,43.662301,-79.389494
M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,43.657162,-79.378937
M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,43.651494,-79.375418
M4E,East Toronto,The Beaches,43.6784,-79.2941,43.676357,-79.293031


In [23]:
!pip install folium

In [26]:
# create map of Toronto using latitude and longitude values of TOronto as the starting point
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


address = 'Toronto, On'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [27]:

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['csv_lat'], df['csv_long'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

# Connecting to foursquare API to segment the Neighborhoods based on venues
We will have to create a foursquare developer credential if you have not done so. 


In [28]:
import foursquare_secrets1

In [29]:
dir(foursquare_secrets1)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'secrets']

In [30]:
CLIENT_ID = foursquare_secrets1.secrets['CLIENT_ID'] # your Foursquare ID
CLIENT_SECRET = foursquare_secrets1.secrets['CLIENT_SECRET'] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [31]:
df.reset_index(inplace=True) # reseting index incase we want to loop over


# Digging into JamesTown

In [32]:
df.loc[1,'Neighborhood']
neighborhood_latitude = df.loc[3,'csv_lat'] # neighborhood latitude value
neighborhood_longitude = df.loc[3,'csv_long'] # neighborhood longitude value

neighborhood_name = df.loc[3,'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. James Town are 43.6514939, -79.3754179.


In [33]:
# type your answer here
radius = 500
LIMIT=100
search_query = ''
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=NSNY0NEA1GEXCEOUCGBGGSEZ5RZX1NNNQEX02S0DUARW0QG4&client_secret=Z2M5ODRGWB5GZT4SCO0WIK1AFTCOQ2HADKXFT5DKR5QMTPO5&ll=43.6514939,-79.3754179&v=20180605&query=&radius=500&limit=100'

In [34]:
results = requests.get(url).json()


In [35]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\trnqu\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
1,GEORGE Restaurant,Restaurant,43.653346,-79.374445
2,Fahrenheit Coffee,Coffee Shop,43.652384,-79.372719
3,Terroni,Italian Restaurant,43.650927,-79.375602
4,Crepe TO,Creperie,43.650063,-79.374587


In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

78 venues were returned by Foursquare.


# Digging into all the Neigborhoods

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
Toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['csv_lat'],
                                   longitudes=df['csv_long']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [40]:
# check the size 
print(Toronto_venues.shape)
Toronto_venues.head()

(1620, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [41]:
#venue by neighborhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [42]:
#unique categories
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


In [43]:
#unique categories
print('There are {} uniques Neighborhoods.'.format(len(Toronto_venues['Neighborhood'].unique())))

There are 39 uniques Neighborhoods.


# Analyizing each neighborhood
one hot encoding will allow all venune categories to be columns

In [44]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")


In [45]:
toronto_onehot['Neighborhood']=Toronto_venues['Neighborhood']  # add neighborhood column back to dataframe

In [46]:
toronto_onehot.columns.get_loc("Neighborhood") # get location of Nieghborhood column


166

In [47]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[166]] + list(toronto_onehot.columns[:166])+ list(toronto_onehot.columns[167:])


In [48]:
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
toronto_onehot.shape

(1620, 237)

In [50]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index() #
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.041667
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.00000,0.015625
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
6,Church and Wellesley,0.013333,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.013333,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013333,0.00000,0.026667
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [51]:
toronto_grouped.shape

(39, 237)

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Restaurant,Café,Gourmet Shop,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Coffee Shop,Breakfast Spot,Yoga Studio,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Skate Park,Brewery,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boutique,Boat or Ferry,Rental Car Location
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Thai Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store


# Cluster Analysis

In [54]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [55]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 1])

In [56]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [58]:
clean_df= df[['Borough', 'Neighborhood', 'csv_lat','csv_long']]

In [59]:
#join the clean df to the foursquare clusters
clean_df=clean_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [60]:
clean_df.head()

,Borough,Neighborhood,csv_lat,csv_long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Pub,Bakery,Café,Theater,Restaurant,Breakfast Spot,Yoga Studio,Shoe Store
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,Sushi Restaurant,Discount Store,Bar,Beer Bar,Smoothie Shop,Sculpture Garden,Sandwich Place,Distribution Center
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Ramen Restaurant,Tea Room
3,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Restaurant,Gastropub,American Restaurant,Cocktail Bar,Department Store,Clothing Store,Creperie,Lingerie Store
4,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Asian Restaurant,Pub,Trail,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Yoga Studio


In [61]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(clean_df['csv_lat'], clean_df['csv_long'], clean_df['Neighborhood'], clean_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
clean_df.loc[clean_df['Cluster Labels'] == 0, clean_df.columns[[1] + list(range(5, clean_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,Health Food Store,Asian Restaurant,Pub,Trail,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Yoga Studio


In [64]:
clean_df.loc[clean_df['Cluster Labels'] == 1, clean_df.columns[[1] + list(range(5, clean_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Bus Line,Swim School,Lawyer,College Rec Center,College Auditorium,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
20,Davisville North,Park,Convenience Store,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Hotel,Gym,Pizza Place,Deli / Bodega
21,"Forest Hill North & West, Forest Hill Road Park",Trail,Park,Bus Line,Jewelry Store,Sushi Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [65]:
clean_df.loc[clean_df['Cluster Labels'] == 2, clean_df.columns[[1] + list(range(5, clean_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Pub,Bakery,Café,Theater,Restaurant,Breakfast Spot,Yoga Studio,Shoe Store
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Sushi Restaurant,Discount Store,Bar,Beer Bar,Smoothie Shop,Sculpture Garden,Sandwich Place,Distribution Center
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Italian Restaurant,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Ramen Restaurant,Tea Room
3,St. James Town,Café,Coffee Shop,Restaurant,Gastropub,American Restaurant,Cocktail Bar,Department Store,Clothing Store,Creperie,Lingerie Store
5,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Restaurant,Café,Gourmet Shop,Pub
6,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Thai Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store
7,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Baby Store,Nightclub,Restaurant,Diner,Athletics & Sports
8,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Hotel,Thai Restaurant,Cosmetics Shop,Pizza Place,Bookstore
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Bank,Middle Eastern Restaurant,Coffee Shop,Music Venue,Brewery,Grocery Store,Park,Supermarket
10,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Restaurant,Sporting Goods Shop,Fried Chicken Joint,Brewery,Scenic Lookout,Plaza


In [66]:
clean_df.loc[clean_df['Cluster Labels'] == 3, clean_df.columns[[1] + list(range(5, clean_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Garden,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [67]:
clean_df.loc[clean_df['Cluster Labels'] == 4, clean_df.columns[[1] + list(range(5, clean_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Park,Restaurant,Playground,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
33,Rosedale,Park,Playground,Trail,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
